In [7]:
! pip install --upgrade --quiet  langchain-openai tavily-python

In [ ]:
! pip install --upgrade langchain

In [ ]:
import os
import sys
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_6.ipynb')
git_add(file_path)
git_commit('Updated test 6')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

d:\Python-codes\packages\lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:

#___________________-   Chatting with GPT-3.5   -______________________#

loader = WebBaseLoader("https://en.wikipedia.org/wiki/FreeSurfer")  # Load the document from the web
docs = loader.load()  # Extract the content

# Split the document into smaller chunks for processing
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # Each chunk has a max size of 200 characters
    chunk_overlap=50  # Overlapping of 50 characters between chunks for context continuity
)
split_docs = splitter.split_documents(docs)  # Split the documents


embedding = OpenAIEmbeddings()  # Generate embeddings using OpenAI
vector_store = FAISS.from_documents(split_docs, embedding=embedding)  # Store documents in FAISS index

retriever = vector_store.as_retriever(search_kwaargs = {"k": 3})  # Convert vector store into a retriever


model = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature=0.5
)

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a friendly assisstant called gyris'),
    MessagesPlaceholder(variable_name = "chat_history"),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name = "agent_scratchpad")
])

# tool for our document 
retriever_tool = create_retriever_tool(
    retriever,
    "toolbox_search",
    "Use this tool when searching for information about FreeSurfer"
    )

# tool for our web search
search = TavilySearchResults() 

# List of tools - we have 2 tools here
tools = [search, retriever_tool]

agent = create_openai_functions_agent(
    llm = model,
    prompt = prompt,
    tools = tools
)

agentExecutor = AgentExecutor(
    agent = agent,
    tools = tools
)

def process_chat(agentExecutor, user_input, chat_history):

    response = agentExecutor.invoke({
    "input": user_input,
    "chat_history": chat_history
    })

    return response["output"]


if __name__ == "__main__":
    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = process_chat(agentExecutor, user_input, chat_history)
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response))
        print("AI: ", response)


AI:  Here are 3 most recent publications about Alzheimer's disease classification:

1. [Improved Approaches for Diagnostic Classification in Alzheimer's Disease](https://pmc.ncbi.nlm.nih.gov/articles/PMC6818723/)
2. [Reviewing Progress Toward Improved Alzheimer's Disease Clinical Trials](https://www.sciencedirect.com/science/article/pii/S2213158218303930)
3. [A Novel Hybrid Approach for Early Detection and Classification of Alzheimer's Disease using sMRI](https://www.nature.com/articles/s41598-024-81563-z)
AI:  Here are the 3 most recent publications about Alzheimer's disease classification with their titles, years, and links:

1. Title: Improved Approaches for Diagnostic Classification in Alzheimer's Disease
   Year: 2019
   [Link to Publication](https://pmc.ncbi.nlm.nih.gov/articles/PMC6818723/)

2. Title: Recent publications from the Alzheimer's Disease Neuroimaging Initiative: Reviewing progress toward improved AD clinical trials
   Year: 2017
   [Link to Publication](https://www.s